In [ ]:
import os
#change working directory to google drive climate risk
os.chdir('/content/drive/My Drive/Climate Risk 2')

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.1 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader

def read_and_clean_pdf(path):
  # reading the pdf document
  reader = PdfReader(path)

  #List of cleaned pdf
  cleaned_pdf_pages = []

  number_of_pages = len(reader.pages)

  # printing number of pages in pdf file
  print(f'number of pdf pages: {number_of_pages}')

  for i in range(0, number_of_pages):
    page = reader.pages[i]

    # extracting text from page
    text = page.extract_text()

    #check if there is a footer
    if '           ' in text:
      # Find the index of the long white space indicating the beginning of the footer
      index = text.index('           ')

      # Get all items before the footer
      result = text[:index]

      # Split the received result in words
      words = result.split()

      # Remove the first 9 words 'Guide on climate- related and environmental risks
      # 6 2 '
      result = ' '.join(words[8:])

      cleaned_pdf_pages.append(result)

    else:
        # Split the received result in words
      words = text.split()

      # Remove the first 9 words 'Guide on climate- related and environmental risks
      # 6 2 '
      result = ' '.join(words[8:])

      cleaned_pdf_pages.append(result)

  return cleaned_pdf_pages


In [ ]:
#read pdf file
cleaned_pdf_pages = read_and_clean_pdf('Policies/cleaned_polices/EU_Central_Bank_Guide_on_climate_related_and_environmental_risks.pdf')


number of pdf pages: 41


In [ ]:
for i in range(len(cleaned_pdf_pages)):
  print(f'page {i + 1} : {cleaned_pdf_pages[i]}')

page 1 : 3 Climate -related and environmental risks 3.1 Definitions Climate change and environmental degradation are sources of structural change that affect economic activity and, in turn, the financial system. Climate- related and environmental risks are commonly understood to comprise t wo main risk drivers: • Physical risk refers to the financial impact of a changing climate, including more frequent extreme weather events and gradual changes in climate, as well as of environmental degradation, such as air, water and land pollution, water stress, biodiversity loss and deforestation.10 Physical risk is therefore categorised as “acute” when it arises from extreme events, such as droughts, floods and storms, and “chronic” when it arises from progressive shifts, such as increasing temperatures, sea- level rises, water stress, biodi versity loss, land use change, habitat destruction and resource scarcity.11 This can directly result in, for example, damage to property or reduced productiv

In [ ]:
# Join all strings in cleaned_pdf_pages into a single string
full_text = ' '.join(cleaned_pdf_pages)

full_text

'3 Climate -related and environmental risks 3.1 Definitions Climate change and environmental degradation are sources of structural change that affect economic activity and, in turn, the financial system. Climate- related and environmental risks are commonly understood to comprise t wo main risk drivers: • Physical risk refers to the financial impact of a changing climate, including more frequent extreme weather events and gradual changes in climate, as well as of environmental degradation, such as air, water and land pollution, water stress, biodiversity loss and deforestation.10 Physical risk is therefore categorised as “acute” when it arises from extreme events, such as droughts, floods and storms, and “chronic” when it arises from progressive shifts, such as increasing temperatures, sea- level rises, water stress, biodi versity loss, land use change, habitat destruction and resource scarcity.11 This can directly result in, for example, damage to property or reduced productivity, or 

In [ ]:
# total characters in the file
len(full_text)

98473

## Load the Model : facebook_bart_summary

In [ ]:
# import and initialize the tokenizer and model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSeq2SeqLM.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# max tokens including the special tokens
tokenizer.model_max_length

1024

In [ ]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [ ]:
# number of special tokens
tokenizer.num_special_tokens_to_add()

2

## Chunk up the document by sentences. Each chunk consists of multiple sentences but ensures each chunk token size is less than the model total acceptable token size

In [ ]:
# convert the contents in full text to sentences
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(full_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

230

In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

18

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1014,
 998,
 1010,
 1016,
 1014,
 1013,
 998,
 997,
 1017,
 1022,
 1017,
 993,
 997,
 992,
 1021,
 1022,
 802]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[1015,
 999,
 1011,
 1017,
 1015,
 1014,
 999,
 998,
 1018,
 1023,
 1018,
 994,
 998,
 993,
 1022,
 1023,
 803]

In [ ]:
sum([len(tokenizer(c).input_ids) for c in chunks])

16960

In [ ]:
len(tokenizer.tokenize(full_text))

Token indices sequence length is longer than the specified maximum sequence length for this model (16943 > 1024). Running this sequence through the model will result in indexing errors


16943

In [ ]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

16943

In [ ]:
len(tokenizer.tokenize(full_text))

16943

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
facebook_bart_summary = []
for input in inputs:
  output = model.generate(**input)
  facebook_bart_summary.append(tokenizer.decode(*output, skip_special_tokens=True))

## Summarized Text:

Climate change and environmental degradation are sources of structural change that affect economic activity and, in turn, the financial system. Climate-related and environmental risks are commonly understood to comprise t wo main risk drivers. Physical risk refers to the financial impact of a changing climate. Transition ri sk refers to an institution’s financial loss that can result, directly or indirectly, from the process of adjustment towards a lower-carbon economy.
Transition risk drivers may affect the viability of some business lines and lead to strategic risk for specific business models. An abrupt repricing of securities, for instance due to asset stranding, may reduce the value of banks’ high quality liquid assets, thereby affecting liquidity buffers. Only a small number of institutions have fully incorporated climate- related and environmental risks into their risk management framework.
Only a few institutions include climate- related risks in their stress testing and reverse stress -testing scenarios. Only a minority of institutions’ disclosures are in line with the recommendations by the TC FD. The ECB has observed that a number of institutions are involved i n initiatives that promote broader and more comparable disclosures.
Business strategy is an institution’s principal tool for positioning itself within its business environment. Climate-related and environmental risks may directly impact the effectiveness of institutions’ existing and future strategies. Institutions are expected to take into account up- to-date scientific insights to enhance their understanding of the potential changes to their business environment going forward.
The ECB observed an institution which had integrated the following climate- related and environmental key performance indicators (KPIs) into its strategic framework. The metrics are cascaded down to the business line level (e.g. retail banking, private banking, commercial banking and corporate banking). For each metric, the respective time horizon is set and progress is measured against a base year.
The management body is expected to ensure that the institution adequately embeds climate- related and environmental risks in the overall business strategy and risk management framework. Institutions are expected to have in place a risk appetite framework (RAF) that considers all the material risks to which the institution is exposed. The ECB expects institutions to monitor and report their exposures to climate-related and environmental risk on the basis of their current data and forward- looking estimations.
Several institutions aim to keep the carbon content of their financed energy mix in line with the target of remaining well below 1.5-2°C, as provided for in the 2015 Paris Agreement. Remuneration policies and practices are expected to help foster a long-term approach to managing climate-related and environmental risks.
Institutions are expected to ensure that the first line of defence performs its duties in accordance with any climate- related and environmental policy, procedure or limit. The ECB observed several institutions that have implemented specific measures to promote a risk culture that takes into account climate-related and environmental risks.
Institutions are expected to ensure that the data reporting framew ork for climate-related and environmental risks operates in conjunction with the climate- related and environmental risk metrics set out in their existing RAS and risk management processes. The data reporting framework is also expected to support, where rel evant, the KPIs used to assess the performance of the institution.
The risk management framework should encompass on- balance- sheet risks and off-balance-sheet risks, with appropriate consideration of both financial and non-financial risks. Institutions are expected to document the climate- related and environmental risks considered, in particular, their transmission channels and impact on the risk profile.
Institutions are expected to conduct a proper climate-related and environmental due diligence, both at the inception of a client relationship and on an ongoing basis. The scope and depth of due diligence is expected to be defined in relation to the sector and geographic location in which the client is located. Institutions are advised to ensure compliance with the OECD Guidelines for Multinational Enterprises.
ECB observed that institutions often consider climate- related and environmental risks qualitatively in their credit -granting process. Some institutions are considering or in the process of developing means to incorporate these risk s into their models. One bank is developing climate -informed shadow probabilities of defaults (PDs) to be reported alongside the regular PDs.
Mortgages with a better energy label receive a lower client rate than mortgages with a less energy -efficient label. This differentiation is based on the consideration that a portfolio with higher energy efficiency labels is likely to be less vulnerable to transition risk. Institutions’ loan pricing is expected to reflect the different costs driven by climate- related and environmental risks.
The ECB observed an institution that considers reputational risks arising from environmental, social or governance impacts in its internal capital adequacy assessment process. Institutions are expected to monitor on an ongoing basis the effect of climate- related and environmental factors on their current market risk positions and future investments.
early or late transition), embed plausible consider ations for the related physical outcome. All these aspects should be properly reflected in an institution’s ICAAP. Institutions are expected to consider adopting a longer time horizon for climate- related and environmental risks given the likelihood that they will mostly materialise in the medium to long term.
EU institutions have reached a political agreement to develop an EU -wide classification system, or taxonomy, for sustainable investments. Going forward, institutions subject to the Non- Financial Reporting Directive (NFRD) will be asked to provide further transparency on the extent to which their activities can be regarded as environmentally sustainable.
Business model, policies and due diligence processes, outcomes, risks and risk management and KPIs. Institutions are reminded of the ECB expectations regarding their business model and strategy, governance and ri sk management as set out in this guide. Disclose Scope 1, Scope 2 and, if appropriate, Scope 3 greenhouse gas (GHG) emissions and the related risks.
As disclosure frameworks and the needs of market participants are evolving rapidly in this area, institutions are well -advised to actively improve their disclosures. The ECB observed a bank that provides schematic reference to its alignment wit h the individual TCFD recommendations. This lays out the respective and specific chapters containing its disclosures that are in line with the TC FD recommendations.

## Load the Model: google pegasus


In [ ]:
from transformers import AutoTokenizer, PegasusModel

model = "google/pegasus-large"

tokenizer = AutoTokenizer.from_pretrained(model)
model = PegasusModel.from_pretrained(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusModel were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

17

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration

model_name = "google/pegasus-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

pegasus_summary = []
for input in inputs:
  input_ids = tokenizer.encode(input, return_tensors='pt')
  output = model.generate(input_ids)
  pegasus_summary.append(tokenizer.decode(output[0], skip_special_tokens=True))

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load Pegasus model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

for input_text in chunks:
  # Tokenize input text
  input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

  # Generate summary
  summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print("Abstractive Summary:")
  print(summary)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Abstractive Summary:
Climate- related and environmental risks are commonly understood to comprise t wo main risk drivers: • Physical risk refers to the financial impact of a changing climate, including more frequent extreme weather events and gradual changes in climate, as well as of environmental degradation, such as air, water and land pollution, water stress, biodiversity loss and deforestation.10 Physical risk is therefore categorised as “acute” when it arises from extreme events, such as droughts, floods and storms, and “chronic” when it arises from progressive shifts, such as increasing temperatures, sea- level rises, water stress, biodi versity loss, land use change, habitat destruction and resource scarcity.11 This can directly result in, for example, damage to property or reduced productivity, or indirectly lead to subsequent events, such as the disruption of supply chains.
Abstractive Summary:
Although the majority of studies have focused on climate- related risks, such as th

Climate- related and environmental risks are commonly understood to comprise t wo main risk drivers: • Physical risk refers to the financial impact of a changing climate, including more frequent extreme weather events and gradual changes in climate, as well as of environmental degradation, such as air, water and land pollution, water stress, biodiversity loss and deforestation.10 Physical risk is therefore categorised as “acute” when it arises from extreme events, such as droughts, floods and storms, and “chronic” when it arises from progressive shifts, such as increasing temperatures, sea- level rises, water stress, biodi versity loss, land use change, habitat destruction and resource scarcity.11 This can directly result in, for example, damage to property or reduced productivity, or indirectly lead to subsequent events, such as the disruption of supply chains. Although the majority of studies have focused on climate- related risks, such as the decline in asset values across carbon- intensive sectors, other environmental factors related to the loss of ecosystem services, such as water stress, biodiversity loss and resource scarcity, have also been shown to drive financial risk.18 19 There is also evidence of an interconnection between climate- related change and environmental risks, resulting in combined effects capable of potentially generat ing even greater impacts.20 Climate- related and environmental risks have distinctive characteristics that warrant particular consideration by supervisors and institutions alike. In this respect, it is important for institutions to identify, assess and monitor the current and forward- looking impact of climate- related and environmental factors on their business environment and to ensure the sustainability and resilience of their business model going forward. The business environment captures a broad range of external factors and trends that shape the business conditions in which an institution operates or is likely to operate based on its main or structural geographic and business exposures.31 These include macroeconomic variables, the competitive landscape, policy and regulation, technology, societal/demographic developments, and geopolitical trends.32 Climate- related and environmental risks may affect all of these areas.
Climate -related and environmental risks may directly impact the effectiveness of institutions’ existing and future strategies.40 Institutions are expected to determine whi ch climate- related and environmental risks impact their business strategy in the short, medium and long term, for example by using (stress) scenario analyses.41 As set out in the EBA Guidelines, institutions should take the limitations, vulnerabilities and shortcomings detected in internal stress tests and scenario analyses into account when determining their business strategy.42 The scenario analysis tool is particularly useful in the context of climate -related and environmental risks given the uncertainty associated with the future course of climate change and society’s response to it.43 By developing a set of plausible scenarios to test the resilience of its business model, institutions can account for these uncertainties in its strategic decision- making.
5 Supervisory expectations relating to governance and risk appetite Under Article 74 of the CRD, institutions are required to have robust governance arrangements in place that allow institutions to effectively identify, manage, monitor and report the risks they are or might be exposed to, so as to form a holistic view of all risks both on an individual and consolidated basis.47 To enable institutions to understand and respond to climate- related and environmental risks, institutions are expected to embed these risks in their governance and risk appetite frameworks, while adequately involving all relevant functions. As set out in the EBA Guidelines, the management body is expected to consider climate and environmental risks in the context of the institution’s overall business strategy, which is expected to provide the management body with the knowledge and expertise to ensure that the institution’s climate and environmental risks are properly managed.59 The management body is expected to be involved in the process of establishing a climate and environmental committee, which draws on the knowledge and expertise of the institution’s members in the context of the institution’s overall business strategy, as well as the knowledge and experience of its members in the context of the institution’s overall business strategy.
The risk inventory is the result of the risk identification process and is expected to be based on the institution’s internal risk taxonomy, which is a categorisation of different risk types and factors within which climate -related and environmental risks are expected to be clearly defined.67 The risk inventory is used as a basis for the formulation of the RAS. In accordance with Article 74 of the CRD, and as further specified in the EBA Guidelines, institutions should have in place a clear, transparent and documented decision- making process and a clear allocation of responsibilities and authority within their internal control framework, including their business lines, internal units and internal control functions.72 that promote informed decision- making by the management body.73 Consequently, responsibilities for identifying, assessing and managing climate- related and environmental risks are expected to be evenly distributed across the different functions within the institution. Institutions may allocate such responsibilit ies across existing stru ctures or, if deem ed useful, may consider establishing a dedicated structure responsible for coordinating the institution’s overall risk management approach to climate- related and environmental risk. Institutions may allocate such responsibilit ies across existing stru ctures or, if deem ed useful, may consider establishing a dedicated structure responsible for coordinating the institution’s overall risk management approach to climate- related and environmental risk. The compliance function should advise the management body on measures to be taken to ensure compliance with applicable laws, rules, regulations and standards, and should assess the possible impact of any changes in the legal or regulatory environment on t he institution’s activities and compliance framework.76 As rules and standards on sustainability may change over time, institutions may increasingly face compliance- related risks, such as liability, litigation and/or reputational risks, stemming from climat e-related and environmental issues. The internal audit function should rev iew the institution’s internal control and risk management framework, by considering external developments, changes in the risk profile and in products and/or business lines, among other things.77 This is expected to include the appropriateness of the arrangements for managing climate -related and environmental risks. An institution’s risk reports are expected to convey the impact of climate- related and environmental risks on its business model, strategy and risk profile.81 Institutions should strive to cover all material climate- related and environmental risks across the legal entity and/or business lines in its risk reports. Institutions are expected to be adaptable in order to generate agg regated climate- related and environmental risk data to meet a broad range of on- demand and ad hoc reporting requests, including requests during stress/crisis situations, requests related to changing internal needs and requests to meet supervisory queries, as demand for climate- related and environmental risks reporting increases. Institutions are expected to adequately quantify the climate- related and environmental risks that the institution is exposed to.89 As also stated in the ECB Guide to the ICAAP, risks are not expected to be excluded from the assessment because they are difficult to quantify or because the relevant data are not available.90 Where such methodologies are subject to further developments, also taking into account the current work and upcoming publications of international networks91 and standard setters, institutions may use plausible assumptions to develop proxies for the assessment of climate- related and environmental risks. Institutions are expected to perform regular internal reviews,97 for instance, in the context of the ICAAP.98 The purpose of these reviews is to assess whether internal processes and methodologies have led to sound outcomes and whether they remain appropriate in view of current and future developments.99 As the availability of data and methodologies for identifying and measuring climate - related and environmental risks is evolving rapidly, institutions are expected to regularly consider the appropriateness and quality of the data sources and methods in place. 6.2 Credit risk management Expectation 8 In their credit risk management, instituti ons are expected to consider climate- related and environmental risks at all relevant stages of the credit -granting process and to monitor the risks in their portfolios. Institutions are expected to incorporate these considerations into both the process for establishing the value of collateral and into the review process prescribed by the applicable regulations.105 Institutions are expected to monitor and manage credit risks in their portfolios, in particular through sectoral/geographic/single -name concentration analysis, including credit risk concentrations stemming from climate- related and environmental risks, and using exposure limits or deleveraging strategies.106 Institutions are expected to monitor how geographic and sectoral concentration is prone to climate- related and environmental risks. Institu tions’ loan pricing frameworks are expected to reflect their credit risk appetite and business strategy with regard to climate -related and environmental risks.107 Pursuant to Article 76(3) of CR D IV, an institution’s risk committee shall review whether the prices of assets offered to clients take the business model and risk strategy fully into account. In particular, when assessing its critical or important functions, an institution is exks pected to consider the impacts of physi cal risks on the provision of these services.112 The outcome of this assessment, if material for any of the institution’s business lines or operations, is expected to be reflected in its business continuity plan. Institutions are expected to evaluate the extent to which the nature of the activities in which they are involved increases the risk of a negative financial impact arising from future reputational damage, liability and/or litigation. Based on the EBA Guidelines, all relevant risks should be encompassed in the risk management framework of an institution, with appropriate consideration of both financial and non- financial risks, including reputational risk.113 Reputational risks can arise quickly and can rapidly affect firms. As part of the ICAAP, institutions are expected to conduct a tailored and in- depth review of their vulnerabilities through stress testing.117 The stress scenarios should compri se all material risks that may deplete internal capital or impact regulatory capital ratios and be used as part of the institution’s stress -testing programme in both the economic and normative perspective. To ensure robust liquidity risk management, institutions are expected to consider the direct or indirect impacts of climate- related and environmental risks on their liquidity position.122 123 Institutions are encouraged to include such considerations in their ILAAP, considering climate- related and environmental risks under both the economic and the normative perspective. Article 432 of the CRD provides that institutions are reminded to take into account in their internal pricing process the specific marginal cost of funding of sustainable refinancing instruments, including, where relevant, the liquidity cost or benefi t compared to ordinary refinancing instruments.124 7 Supervisory expectations relating to disclosures Access to information is necessary for promoting transparency within financial institutions and contributing to the orderly functioning of financial markets.125 The European regulatory framework therefore sets out disclosure requirements to make key information available about an institution’s capital, risks and risk exposures in order to adequately inform market participants. Furthermore, the EBA Guidelines state that, when an institution decides not to disclose information or a set of requirements due to immateriality, it should clearly state this fact.135 When institutions disclose figures, metrics and targets as material, they are expected to disclose or reference the methodologies, definitions and criteria associated with them.136 These disclosures help to convey the institution’ s risk profile in a comprehensive manner to market participants, namely with a view to limiting reputational and liability risks. In line with the European Commission’s Supplement and the EBA’s key policy messages, institutions are expected to disclose the metrics used, including relevant target s and the current performance of the institution against those targets. Using the aforementioned metrics, the institution is expected to describe the short, medium and long- term resilience of its strategy and risk management processes in the light of different climate- related scenarios.


In [ ]:
summarized_text = 'Climate change and environmental degradation are sources of structural change that affect economic activity and, in turn, the financial system. Climate-related and environmental risks are commonly understood to comprise t wo main risk drivers. Physical risk refers to the financial impact of a changing climate. Transition ri sk refers to an institution’s financial loss that can result, directly or indirectly, from the process of adjustment towards a lower-carbon economy. Transition risk drivers may affect the viability of some business lines and lead to strategic risk for specific business models. An abrupt repricing of securities, for instance due to asset stranding, may reduce the value of banks’ high quality liquid assets, thereby affecting liquidity buffers. Only a small number of institutions have fully incorporated climate- related and environmental risks into their risk management framework. Only a few institutions include climate- related risks in their stress testing and reverse stress -testing scenarios. Only a minority of institutions’ disclosures are in line with the recommendations by the TC FD. The ECB has observed that a number of institutions are involved i n initiatives that promote broader and more comparable disclosures. Business strategy is an institution’s principal tool for positioning itself within its business environment. Climate-related and environmental risks may directly impact the effectiveness of institutions’ existing and future strategies. Institutions are expected to take into account up- to-date scientific insights to enhance their understanding of the potential changes to their business environment going forward. The ECB observed an institution which had integrated the following climate- related and environmental key performance indicators (KPIs) into its strategic framework. The metrics are cascaded down to the business line level (e.g. retail banking, private banking, commercial banking and corporate banking). For each metric, the respective time horizon is set and progress is measured against a base year. The management body is expected to ensure that the institution adequately embeds climate- related and environmental risks in the overall business strategy and risk management framework. Institutions are expected to have in place a risk appetite framework (RAF) that considers all the material risks to which the institution is exposed. The ECB expects institutions to monitor and report their exposures to climate-related and environmental risk on the basis of their current data and forward- looking estimations. Several institutions aim to keep the carbon content of their financed energy mix in line with the target of remaining well below 1.5-2°C, as provided for in the 2015 Paris Agreement. Remuneration policies and practices are expected to help foster a long-term approach to managing climate-related and environmental risks. Institutions are expected to ensure that the first line of defence performs its duties in accordance with any climate- related and environmental policy, procedure or limit. The ECB observed several institutions that have implemented specific measures to promote a risk culture that takes into account climate-related and environmental risks. Institutions are expected to ensure that the data reporting framew ork for climate-related and environmental risks operates in conjunction with the climate- related and environmental risk metrics set out in their existing RAS and risk management processes. The data reporting framework is also expected to support, where rel evant, the KPIs used to assess the performance of the institution. The risk management framework should encompass on- balance- sheet risks and off-balance-sheet risks, with appropriate consideration of both financial and non-financial risks. Institutions are expected to document the climate- related and environmental risks considered, in particular, their transmission channels and impact on the risk profile. Institutions are expected to conduct a proper climate-related and environmental due diligence, both at the inception of a client relationship and on an ongoing basis. The scope and depth of due diligence is expected to be defined in relation to the sector and geographic location in which the client is located. Institutions are advised to ensure compliance with the OECD Guidelines for Multinational Enterprises. ECB observed that institutions often consider climate- related and environmental risks qualitatively in their credit -granting process. Some institutions are considering or in the process of developing means to incorporate these risk s into their models. One bank is developing climate -informed shadow probabilities of defaults (PDs) to be reported alongside the regular PDs. Mortgages with a better energy label receive a lower client rate than mortgages with a less energy -efficient label. This differentiation is based on the consideration that a portfolio with higher energy efficiency labels is likely to be less vulnerable to transition risk. Institutions’ loan pricing is expected to reflect the different costs driven by climate- related and environmental risks. The ECB observed an institution that considers reputational risks arising from environmental, social or governance impacts in its internal capital adequacy assessment process. Institutions are expected to monitor on an ongoing basis the effect of climate- related and environmental factors on their current market risk positions and future investments. early or late transition), embed plausible consider ations for the related physical outcome. All these aspects should be properly reflected in an institution’s ICAAP. Institutions are expected to consider adopting a longer time horizon for climate- related and environmental risks given the likelihood that they will mostly materialise in the medium to long term. EU institutions have reached a political agreement to develop an EU -wide classification system, or taxonomy, for sustainable investments. Going forward, institutions subject to the Non- Financial Reporting Directive (NFRD) will be asked to provide further transparency on the extent to which their activities can be regarded as environmentally sustainable. Business model, policies and due diligence processes, outcomes, risks and risk management and KPIs. Institutions are reminded of the ECB expectations regarding their business model and strategy, governance and ri sk management as set out in this guide. Disclose Scope 1, Scope 2 and, if appropriate, Scope 3 greenhouse gas (GHG) emissions and the related risks. As disclosure frameworks and the needs of market participants are evolving rapidly in this area, institutions are well -advised to actively improve their disclosures. The ECB observed a bank that provides schematic reference to its alignment wit h the individual TCFD recommendations. This lays out the respective and specific chapters containing its disclosures that are in line with the TC FD recommendations.'

In [ ]:
# convert the contents in full text to sentences
import nltk
nltk.download('punkt')
sum_sentences = nltk.tokenize.sent_tokenize(summarized_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sum_sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sum_sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

2

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load Pegasus model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

for input_text in chunks:
  # Tokenize input text
  input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

  # Generate summary
  summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print("Abstractive Summary:")
  print(summary)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Abstractive Summary:
Transition risk drivers may affect the viability of some business lines and lead to strategic risk for specific business models. The ECB expects institutions to monitor and report their exposures to climate-related and environmental risk on the basis of their current data and forward- looking estimations.
Abstractive Summary:
Institutions are reminded of the ECB expectations regarding their business model and strategy, governance and ri sk management as set out in this guide.


Transition risk drivers may affect the viability of some business lines and lead to strategic risk for specific business models. The ECB expects institutions to monitor and report their exposures to climate-related and environmental risk on the basis of their current data and forward- looking estimations. Institutions are reminded of the ECB expectations regarding their business model and strategy, governance and risk management as set out in this guide.